# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.3 MB 12 kB/s 
     |████████████████████████████████| 394.3 MB 15 kB/s 
     |████████████████████████████████| 32.9 MB 1.4 MB/s 
     |████████████████████████████████| 462 kB 49.0 MB/s 
     |████████████████████████████████| 132 kB 47.5 MB/s 
     |████████████████████████████████| 14.8 MB 35.1 MB/s 
     |████████████████████████████████| 3.8 MB 32.9 MB/s 
     |████████████████████████████████| 2.9 MB 27.1 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68722 sha256=2be1468f67c7665821a5a6832f5024d6fea5e16fdbdd6d192aa11143324e7235
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
   

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image,model):

	image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
	image.flags.writeable=False
	results=model.process(image)
	image.flags.writeable=True
	image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
	return frame,results


In [ ]:
def draw_landmarks(image,results):
	mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
	mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
	mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
	mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, #
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
#TEST WITH CAMERA - DO NOT RUN 
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():bgddvfv

        # Read feedbg
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
draw_landmarks(frame, results)

# 3. Extract Keypoint Values

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
result_test = extract_keypoints(results)

NameError: ignored

In [ ]:
result_test

array([ 0.53810918,  0.49632657, -0.9924382 , ...,  0.        ,
        0.        ,  0.        ])

# 4. Setup Folders for Collection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
""" # Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])
 """
# Load the data
DATA_OUT='/content/drive/Shareddrives/TALOV/Data_out_{n}'
DATA_IN = '/content/drive/Shareddrives/TALOV/ASL_Augmented'
#/content/drive/Shareddrives/TALOV/ASL Source
# Six hundred videos worth of data
no_sequences = 600

# Videos are going to be 30 frames in length
sequence_length = 30



In [ ]:
##List clases in folder Data/In
from numpy import random

#actions2 = os.listdir(DATA_IN)
#list_actions = random.randint(len(actions2), size=(12))
actions = ["Hard","Hot","House","Interpreter","Keep","Last","Late","Learn","Leave"]
#for action in list_actions:

#  actions.append(actions2[action])

#print(actions)

##List images in folder Data/In
list_images = []
for classs in actions:
    print(classs)
    image = os.listdir(DATA_IN + '/' + classs) #worse
    list_images.append(image)
    
    

#for k in range(12):
#    print(list_images[k], len(list_images[k]))



Hard
Hot
House
Interpreter
Keep
Last
Late
Learn
Leave


In [ ]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_OUT, action, str(sequence)))
        except:
            pass

In [ ]:
#CREATING FOLDERS DO NOT RUN UNLESS YOU DONT WANT TO CREATE FOLDERS
#for action in actions: 
 #   dirmax = np.max(np.array(os.listdir(os.path.join(DATA_OUT, action))).astype(int))
  #  for sequence in range(1,no_sequences+1):
   #     try: 
    #        os.makedirs(os.path.join(DATA_OUT, action, str(dirmax+sequence)))
     #   except:
      #      pass
            

# 5. Collect Keypoint Values for Training and Testing

In [ ]:

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    cont=0
    ct=0
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        path = DATA_IN + '/' + action +'/'
        videos = os.listdir(path)
        
        for sequence in range(no_sequences):
            ct+=1
            # Loop through video length aka sequence length
            cap = cv2.VideoCapture( path + videos[sequence])
            for frame_num in range(sequence_length):
                
                if int(cap.get(cv2.CAP_PROP_FPS)) < 30:
                  cont+=1
                
                if int(cap.get(cv2.CAP_PROP_FPS)) > 30:
                  ret, frame = cap.read()
                  print(cap.get(cv2.CAP_PROP_FPS), "FPS")

                  # Make detections
                  image, results = mediapipe_detection(frame, holistic)
                  print(sequence, "SEQUENCE", action, "ACTION", frame_num, "frameNum")

                  keypoints = extract_keypoints(results)
                  npy_path = os.path.join(DATA_OUT, action, str(sequence), str(frame_num))
                  np.save(npy_path, keypoints)

                  """ length   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                  fps_rate = int(cap.get(cv2.CAP_PROP_FPS))
                  print(fps_rate)
                  print( length ) """

                
                # NEW Export keypoints
            print("La cantidad de acciones de", sequence, "es:",ct)   

                # Break gracefully
                #if cv2.waitKey(10) & 0xFF == ord('q'):
                #    break
    print("Cantidad de videos no tomados por no llegar a 30 fps:",cont)                
    cap.release()
      cv2.destroyAllWindows()

Se truncaron las últimas líneas 5000 del resultado de transmisión.
60.0 FPS
508 SEQUENCE House ACTION 16 frameNum
60.0 FPS
508 SEQUENCE House ACTION 17 frameNum
60.0 FPS
508 SEQUENCE House ACTION 18 frameNum
60.0 FPS
508 SEQUENCE House ACTION 19 frameNum
60.0 FPS
508 SEQUENCE House ACTION 20 frameNum
60.0 FPS
508 SEQUENCE House ACTION 21 frameNum
60.0 FPS
508 SEQUENCE House ACTION 22 frameNum
60.0 FPS
508 SEQUENCE House ACTION 23 frameNum
60.0 FPS
508 SEQUENCE House ACTION 24 frameNum
60.0 FPS
508 SEQUENCE House ACTION 25 frameNum
60.0 FPS
508 SEQUENCE House ACTION 26 frameNum
60.0 FPS
508 SEQUENCE House ACTION 27 frameNum
60.0 FPS
508 SEQUENCE House ACTION 28 frameNum
60.0 FPS
508 SEQUENCE House ACTION 29 frameNum
La cantidad de acciones de 508 es: 1709
60.0 FPS
509 SEQUENCE House ACTION 0 frameNum
60.0 FPS
509 SEQUENCE House ACTION 1 frameNum
60.0 FPS
509 SEQUENCE House ACTION 2 frameNum
60.0 FPS
509 SEQUENCE House ACTION 3 frameNum
60.0 FPS
509 SEQUENCE House ACTION 4 frameNum
60.0 

OSError: ignored

In [ ]:
#cap.release()
#cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

{'Hard': 0,
 'Hot': 1,
 'House': 2,
 'Interpreter': 3,
 'Keep': 4,
 'Last': 5,
 'Late': 6,
 'Learn': 7,
 'Leave': 8}

In [ ]:
sequences, labels = [], []
contaa=0
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_OUT, action))).astype(int):
        window = []
        path = os.path.join(DATA_OUT, action, str(sequence))
        print(sequence,action)
        dir = os.listdir(path)
        for frame_num in range(sequence_length):
          if len(dir) != 0: 
            res = np.load(os.path.join(DATA_OUT, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
          else: 
            contaa+=1
    
        sequences.append(window)
        labels.append(label_map[action])
    print("Carpetas vacias",contaa) 

NameError: ignored

In [ ]:
###################################################################
#sequences, labels = [], []
#for action in actions:
 #   for sequence in np.array(os.listdir(os.path.join(DATA_OUT, action))).astype(int):
  #      window = []
   #     print(sequence)
    #    for frame_num in range(sequence_length):
     #     
      #      res = np.load(os.path.join(DATA_OUT, action, str(sequence), "{}.npy".format(frame_num)))
       ###     window.append(res)
        #sequences.append(window)
        #labels.append(label_map[action])

        #########################################

In [ ]:
np.save('sequencesNotebook.npy', sequences)

In [ ]:
np.save('labelsNotebook.npy', labels)

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape

# 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
6/6 [==============================] - 4s 87ms/step - loss: 1.8267 - categorical_accuracy: 0.1462
Epoch 2/1000
6/6 [==============================] - 1s 88ms/step - loss: 1.8203 - categorical_accuracy: 0.2105
Epoch 3/1000
6/6 [==============================] - 1s 84ms/step - loss: 1.6683 - categorical_accuracy: 0.2690
Epoch 4/1000
6/6 [==============================] - 1s 86ms/step - loss: 1.4748 - categorical_accuracy: 0.3743
Epoch 5/1000
6/6 [==============================] - 1s 85ms/step - loss: 1.3680 - categorical_accuracy: 0.2982
Epoch 6/1000
6/6 [==============================] - 1s 87ms/step - loss: 1.1095 - categorical_accuracy: 0.4035
Epoch 7/1000
6/6 [==============================] - 1s 87ms/step - loss: 1.1896 - categorical_accuracy: 0.4444
Epoch 8/1000
6/6 [==============================] - 1s 89ms/step - loss: 1.2280 - categorical_accuracy: 0.4620
Epoch 9/1000
6/6 [==============================] - 1s 90ms/step - loss: 1.2144 - categorical_accuracy: 0.3450
E

6/6 [==============================] - 1s 105ms/step - loss: 1.2987 - categorical_accuracy: 0.3684
Epoch 75/1000
6/6 [==============================] - 1s 103ms/step - loss: 1.0818 - categorical_accuracy: 0.4035
Epoch 76/1000
6/6 [==============================] - 1s 108ms/step - loss: 0.9715 - categorical_accuracy: 0.4737
Epoch 77/1000
6/6 [==============================] - 1s 100ms/step - loss: 0.9866 - categorical_accuracy: 0.4620
Epoch 78/1000
6/6 [==============================] - 1s 105ms/step - loss: 0.9627 - categorical_accuracy: 0.4854
Epoch 79/1000
6/6 [==============================] - 1s 96ms/step - loss: 0.9232 - categorical_accuracy: 0.4971
Epoch 80/1000
6/6 [==============================] - 1s 94ms/step - loss: 0.8849 - categorical_accuracy: 0.5029
Epoch 81/1000
6/6 [==============================] - 1s 97ms/step - loss: 0.8584 - categorical_accuracy: 0.5263
Epoch 82/1000
6/6 [==============================] - 1s 95ms/step - loss: 0.8434 - categorical_accuracy: 0.5848
E

6/6 [==============================] - 1s 110ms/step - loss: 0.9536 - categorical_accuracy: 0.5673
Epoch 147/1000
6/6 [==============================] - 1s 105ms/step - loss: 0.8517 - categorical_accuracy: 0.5965
Epoch 148/1000
6/6 [==============================] - 1s 109ms/step - loss: 0.8074 - categorical_accuracy: 0.5497
Epoch 149/1000
6/6 [==============================] - 1s 121ms/step - loss: 0.9532 - categorical_accuracy: 0.4620
Epoch 150/1000
6/6 [==============================] - 1s 96ms/step - loss: 0.8119 - categorical_accuracy: 0.5614
Epoch 151/1000
6/6 [==============================] - 1s 105ms/step - loss: 0.7980 - categorical_accuracy: 0.5497
Epoch 152/1000
6/6 [==============================] - 1s 115ms/step - loss: 0.7666 - categorical_accuracy: 0.6082
Epoch 153/1000
6/6 [==============================] - 1s 145ms/step - loss: 0.8348 - categorical_accuracy: 0.5614
Epoch 154/1000
6/6 [==============================] - 1s 104ms/step - loss: 0.7520 - categorical_accurac

6/6 [==============================] - 1s 96ms/step - loss: 0.3085 - categorical_accuracy: 0.8713
Epoch 219/1000
6/6 [==============================] - 1s 106ms/step - loss: 0.3921 - categorical_accuracy: 0.7953
Epoch 220/1000
6/6 [==============================] - 1s 94ms/step - loss: 0.4042 - categorical_accuracy: 0.8129
Epoch 221/1000
6/6 [==============================] - 1s 92ms/step - loss: 0.4855 - categorical_accuracy: 0.7661
Epoch 222/1000
6/6 [==============================] - 1s 93ms/step - loss: 0.5289 - categorical_accuracy: 0.7719
Epoch 223/1000
6/6 [==============================] - 1s 102ms/step - loss: 0.5399 - categorical_accuracy: 0.7193
Epoch 224/1000
6/6 [==============================] - 1s 94ms/step - loss: 0.4206 - categorical_accuracy: 0.7953
Epoch 225/1000
6/6 [==============================] - 1s 93ms/step - loss: 0.3613 - categorical_accuracy: 0.8129
Epoch 226/1000
6/6 [==============================] - 1s 92ms/step - loss: 0.3147 - categorical_accuracy: 0.8

6/6 [==============================] - 1s 95ms/step - loss: 9.5145e-04 - categorical_accuracy: 1.0000
Epoch 291/1000
6/6 [==============================] - 1s 94ms/step - loss: 8.6372e-04 - categorical_accuracy: 1.0000
Epoch 292/1000
6/6 [==============================] - 1s 92ms/step - loss: 8.1671e-04 - categorical_accuracy: 1.0000
Epoch 293/1000
6/6 [==============================] - 1s 97ms/step - loss: 7.9372e-04 - categorical_accuracy: 1.0000
Epoch 294/1000
6/6 [==============================] - 1s 94ms/step - loss: 7.1465e-04 - categorical_accuracy: 1.0000
Epoch 295/1000
6/6 [==============================] - 1s 93ms/step - loss: 6.6526e-04 - categorical_accuracy: 1.0000
Epoch 296/1000
6/6 [==============================] - 1s 95ms/step - loss: 6.4914e-04 - categorical_accuracy: 1.0000
Epoch 297/1000
6/6 [==============================] - 1s 95ms/step - loss: 6.0648e-04 - categorical_accuracy: 1.0000
Epoch 298/1000
6/6 [==============================] - 1s 94ms/step - loss: 5.77

...

6/6 [==============================] - 1s 93ms/step - loss: 2.6156e-06 - categorical_accuracy: 1.0000
Epoch 710/1000
6/6 [==============================] - 1s 93ms/step - loss: 2.6031e-06 - categorical_accuracy: 1.0000
Epoch 711/1000
6/6 [==============================] - 1s 93ms/step - loss: 2.5968e-06 - categorical_accuracy: 1.0000
Epoch 712/1000
6/6 [==============================] - 1s 92ms/step - loss: 2.5801e-06 - categorical_accuracy: 1.0000
Epoch 713/1000
6/6 [==============================] - 1s 93ms/step - loss: 2.5682e-06 - categorical_accuracy: 1.0000
Epoch 714/1000
6/6 [==============================] - 1s 94ms/step - loss: 2.5550e-06 - categorical_accuracy: 1.0000
Epoch 715/1000
6/6 [==============================] - 1s 99ms/step - loss: 2.5403e-06 - categorical_accuracy: 1.0000
Epoch 716/1000
6/6 [==============================] - 1s 96ms/step - loss: 2.5292e-06 - categorical_accuracy: 1.0000
Epoch 717/1000
6/6 [==============================] - 1s 91ms/step - loss: 2.51

6/6 [==============================] - 1s 93ms/step - loss: 1.8850e-06 - categorical_accuracy: 1.0000
Epoch 780/1000
6/6 [==============================] - 1s 92ms/step - loss: 1.8760e-06 - categorical_accuracy: 1.0000
Epoch 781/1000
6/6 [==============================] - 1s 92ms/step - loss: 1.8697e-06 - categorical_accuracy: 1.0000
Epoch 782/1000
6/6 [==============================] - 1s 95ms/step - loss: 1.8613e-06 - categorical_accuracy: 1.0000
Epoch 783/1000
6/6 [==============================] - 1s 93ms/step - loss: 1.8544e-06 - categorical_accuracy: 1.0000
Epoch 784/1000
6/6 [==============================] - 1s 93ms/step - loss: 1.8453e-06 - categorical_accuracy: 1.0000
Epoch 785/1000
6/6 [==============================] - 1s 94ms/step - loss: 1.8362e-06 - categorical_accuracy: 1.0000
Epoch 786/1000
6/6 [==============================] - 1s 94ms/step - loss: 1.8279e-06 - categorical_accuracy: 1.0000
Epoch 787/1000
6/6 [==============================] - 1s 93ms/step - loss: 1.81

6/6 [==============================] - 1s 94ms/step - loss: 1.3998e-06 - categorical_accuracy: 1.0000
Epoch 850/1000
6/6 [==============================] - 1s 92ms/step - loss: 1.3950e-06 - categorical_accuracy: 1.0000
Epoch 851/1000
6/6 [==============================] - 1s 93ms/step - loss: 1.3880e-06 - categorical_accuracy: 1.0000
Epoch 852/1000
6/6 [==============================] - 1s 90ms/step - loss: 1.3803e-06 - categorical_accuracy: 1.0000
Epoch 853/1000
6/6 [==============================] - 1s 92ms/step - loss: 1.3754e-06 - categorical_accuracy: 1.0000
Epoch 854/1000
6/6 [==============================] - 1s 90ms/step - loss: 1.3719e-06 - categorical_accuracy: 1.0000
Epoch 855/1000
6/6 [==============================] - 1s 93ms/step - loss: 1.3692e-06 - categorical_accuracy: 1.0000
Epoch 856/1000
6/6 [==============================] - 1s 90ms/step - loss: 1.3601e-06 - categorical_accuracy: 1.0000
Epoch 857/1000
6/6 [==============================] - 1s 90ms/step - loss: 1.35

6/6 [==============================] - 1s 95ms/step - loss: 1.0673e-06 - categorical_accuracy: 1.0000
Epoch 920/1000
6/6 [==============================] - 1s 97ms/step - loss: 1.0589e-06 - categorical_accuracy: 1.0000
Epoch 921/1000
6/6 [==============================] - 1s 96ms/step - loss: 1.0548e-06 - categorical_accuracy: 1.0000
Epoch 922/1000
6/6 [==============================] - 1s 92ms/step - loss: 1.0548e-06 - categorical_accuracy: 1.0000
Epoch 923/1000
6/6 [==============================] - 1s 94ms/step - loss: 1.0506e-06 - categorical_accuracy: 1.0000
Epoch 924/1000
6/6 [==============================] - 1s 93ms/step - loss: 1.0457e-06 - categorical_accuracy: 1.0000
Epoch 925/1000
6/6 [==============================] - 1s 96ms/step - loss: 1.0429e-06 - categorical_accuracy: 1.0000
Epoch 926/1000
6/6 [==============================] - 1s 98ms/step - loss: 1.0387e-06 - categorical_accuracy: 1.0000
Epoch 927/1000
6/6 [==============================] - 1s 92ms/step - loss: 1.03

6/6 [==============================] - 1s 97ms/step - loss: 8.1982e-07 - categorical_accuracy: 1.0000
Epoch 990/1000
6/6 [==============================] - 1s 95ms/step - loss: 8.1634e-07 - categorical_accuracy: 1.0000
Epoch 991/1000
6/6 [==============================] - 1s 93ms/step - loss: 8.1494e-07 - categorical_accuracy: 1.0000
Epoch 992/1000
6/6 [==============================] - 1s 99ms/step - loss: 8.1146e-07 - categorical_accuracy: 1.0000
Epoch 993/1000
6/6 [==============================] - 1s 95ms/step - loss: 8.0937e-07 - categorical_accuracy: 1.0000
Epoch 994/1000
6/6 [==============================] - 1s 94ms/step - loss: 8.0449e-07 - categorical_accuracy: 1.0000
Epoch 995/1000
6/6 [==============================] - 1s 95ms/step - loss: 8.0100e-07 - categorical_accuracy: 1.0000
Epoch 996/1000
6/6 [==============================] - 1s 91ms/step - loss: 7.9891e-07 - categorical_accuracy: 1.0000
Epoch 997/1000
6/6 [==============================] - 1s 91ms/step - loss: 7.94

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596,774
Trainable params: 596,774
Non-trai

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
res

array([[6.9760180e-29, 5.8170661e-21, 3.4253250e-07, 9.5204233e-10,
        2.0445211e-23, 9.9999964e-01],
       [1.4086175e-06, 1.6570194e-19, 1.0916248e-09, 6.0559823e-23,
        9.9999857e-01, 4.5728240e-18],
       [1.8543605e-04, 1.1890131e-23, 1.0565464e-15, 1.2160591e-30,
        9.9981457e-01, 2.6222581e-24],
       [1.3486598e-27, 4.0608719e-10, 1.9516369e-18, 9.9999940e-01,
        6.0883373e-26, 6.3310267e-07],
       [1.2656168e-05, 1.3000889e-19, 5.9598104e-11, 4.3938902e-24,
        9.9998736e-01, 4.6472801e-19],
       [1.2532507e-05, 1.2842491e-23, 2.9791158e-14, 1.2893550e-29,
        9.9998748e-01, 2.3054509e-23],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 1.2575717e-16,
        0.0000000e+00, 2.6817307e-36],
       [9.9999118e-01, 5.5493576e-24, 1.1363258e-20, 2.4513050e-30,
        8.8402157e-06, 1.2253413e-22],
       [2.3146391e-20, 5.9207752e-26, 1.0000000e+00, 4.0049407e-20,
        4.2308606e-08, 1.8481973e-10]], dtype=float32)

In [ ]:
actions[np.argmax(res[6])]

'Yearlong'

In [ ]:
actions[np.argmax(y_test[4])]

'Young'

# 9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

NameError: ignored

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245), (245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [1]:
# plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 

Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediap

Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class '

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yesterday
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'me

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Young
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Young
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Young
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Young
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.Solut

Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yearlong
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yourself
<

Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yellow
